In [2]:
import tensorflow as tf
import struct

In [ ]:
import struct

def index_tfrecord(tfrecord, index_filepath):
    with open(tfrecord, 'rb') as tfr, open(index_filepath, 'w') as idx:
        while True:
            current = tfr.tell()
            try:
                # byte length
                byte_len = tfr.read(8)
                if len(byte_len) == 0:
                    break

                # crc length
                byte_len_crc = tfr.read(4)
                proto_len = struct.unpack('q', byte_len)[0]

                # proto
                tfr.read(proto_len)

                # crc
                tfr.read(4)
                print(str(current) + '\t' + str(tfr.tell() - current), file=idx)
            except Exception:
                print('Not a valid TFRecord file.')
                break

In [ ]:
index_tfrecord('test.tfrecord', 'test.tfrecord.idx')

In [5]:
from bioflow.io import dataset_ops

features = dataset_ops.features_from_json_file('test.tfrecord.features.json')
features

FeaturesDict({
    'inputs': FeaturesDict({
        'sequence': Tensor(shape=(None, 4), dtype=int8),
    }),
    'outputs': FeaturesDict({
        'SomeTask': FeaturesDict({
            'total': Tensor(shape=(None,), dtype=float32),
        }),
    }),
})

In [ ]:
import tensorflow as tf

# with open('test.tfrecord', 'rb') as tfrecord:
with tf.io.gfile.GFile('test.tfrecord', 'rb') as tfrecord:
    tfrecord.seek(tf.constant(780, dtype=tf.int64)) # 780
    record_bytes = tfrecord.read(348)
    proto_bytes = record_bytes[(8+4):-4]
record_bytes

In [ ]:
int(tf.constant(7))

In [6]:
from bioflow.io.dataset_ops import better_py_function_kwargs

class GFileTFRecord:
    def __init__(self, filepath, features=None):
        self._gfile_tfrecord = tf.io.gfile.GFile(filepath, 'rb')
        self.features = features
    
    def _seek_and_read(self, offset, length):
        offset, length = int(offset), int(length)

        self._gfile_tfrecord.seek(offset)
        return self._gfile_tfrecord.read(length)[(8+4):-4]

    def read_proto(self, offset, length):
        proto_bytes = tf.py_function(self._seek_and_read, inp=[offset, length], Tout=tf.string)
        proto_bytes.set_shape(shape=())
        return proto_bytes
    
    def read_example(self, offset, length):
        proto = self.read_proto(offset, length)
        return features.deserialize_example(proto)

gfile_tfrecord = GFileTFRecord('test.tfrecord', features)

In [7]:
proto = gfile_tfrecord.read_proto(780, 348)
proto

2023-02-03 16:02:06.457811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-03 16:02:06.457874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-03 16:02:06.457921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-03 16:02:06.457968: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-03 16:02:06.458015: W tensorfl

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xc9\x02\n\xbc\x01\n\x16outputs/SomeTask/total\x12\xa1\x01\n\x9e\x01\n\x9b\x01x\x9cc`h\xb0g`pp`\x00\xd30\xf6\x02(f\x00b\x03G\x888\x88}\x00I\x1c\x8c\x19\xd0\xd8\xd8\x00>\xb9\x11\x08\xc0a\x89\x03H810\xbc\x03\xe2E\xce\x0c\x0c\xe7\x804\x03\x10\xbf\x00\x86\xff\x04 6\x00\xb2\x0b\x80\xf4\x0e \x16q\x82\xc4\x05(l\x1d\xa04,\xfe\x1a\xa0\xfc\x07 \xda\x11\xc2\x87\xc5+X\x9d\x03\xaa\x1b\xc0\xe60 \xf4\xe3\x02\xc8\xf1\xde\x80\xc4v@b\'8"\xf1\x91\xfc\xeb\x00U\xd7\x00u\x17\xdc\xad\xf6\xa8j\x1d\xd0\xdc\x02\xc2\x0f\xd0\xcc\x82\xbb\xd1\x01\xd3\xfd\xb4\x01\x00\x9b\x8b1L\n\x87\x01\n\x0finputs/sequence\x12t\nr\npx\x9c\xadQK\x16\x00!\x08\x92\xfb_z\x16\xe5 \x8a\xbb\xde\xeb\xa3 \x95\x11\x11\xc8\x11g\x85\xc4\xc2m\xf1\xd5l\xdcI |\xbd*qV^(\xe0rj\xc8J\xfd\xa3\x9d-\xf1e\x1b\xbf\xedu:\xcc\xcdq\xb7\x02\xff\xffIjA(\x8e\xf66\x91.>v\x0f\xab\x07j \x8a\x8d\xa2\xb4\xe7\x8c\x9e\x1aV\xab\x1d\xdf\xfc\x18\xf5\xa2m\xfd\xb9\x7f7x\x06\x1f?\x03\x00\xca'>

In [10]:
features.deserialize_example(proto, decoders=None)

{'inputs': {'sequence': <tf.Tensor: shape=(201, 4), dtype=int8, numpy=
  array([[0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [0, 0, 1, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 1],
         

In [ ]:
example = gfile_tfrecord.read_example(780, 348)
example.keys()

In [ ]:
example = gfile_tfrecord.read_example(tf.constant(780, dtype=tf.int64), tf.constant(348, dtype=tf.int64))
example.keys()

In [ ]:
offset_length = []
with open('test.tfrecord.idx') as f:
    for line in f:
        offset, length = line.strip().split()
        offset_length.append([int(offset), int(length)])
print(len(offset_length))

In [3]:
import pandas as pd
import numpy as np

def load_index(filepath):
    return np.array(pd.read_csv(filepath, sep='\t', header=None), dtype=np.int64)

index = load_index('test.tfrecord.idx')
index

array([[     0,    348],
       [   348,    432],
       [   780,    348],
       ...,
       [180570,    249],
       [180819,    372],
       [181191,    283]])

In [ ]:
import tensorflow as tf

def index_to_dataset(filepath):
    index = load_index(filepath)
    dataset = tf.data.Dataset.from_tensor_slices((index[:, 0], index[:, 1]))
    return dataset

idx_dataset = index_to_dataset('test.tfrecord.idx')
print(idx_dataset)
print(next(iter(idx_dataset)))
print(idx_dataset.cardinality())

In [ ]:
proto_dataset = idx_dataset.map(gfile_tfrecord.read_proto).shuffle(np.inf)
print(proto_dataset)
print(next(iter(proto_dataset)))

In [ ]:
idx_dataset = tf.data.Dataset.from_tensor_slices(offset_length)
idx_dataset = idx_dataset.map(lambda x: (tf.cast(x[0], tf.int64), tf.cast(x[1], tf.int64)))
idx_dataset = idx_dataset.repeat(1)
idx_dataset = idx_dataset.shuffle(1_000_000)
print(idx_dataset)
print(next(iter(idx_dataset)))

In [ ]:
proto_dataset = idx_dataset.take(1).map(gfile_tfrecord.read_proto)
proto_dataset

print(next(iter(proto_dataset)))

In [ ]:
example_dataset = idx_dataset.map(gfile_tfrecord.read_example)
example_dataset

In [ ]:
n = 0
for example in example_dataset:
    x = example
    n += 1
print(n)

In [ ]:
features.deserialize_example(proto_bytes)

In [5]:
from bioflow.io import load_indexed_tfrecord

dataset = load_indexed_tfrecord('/home/marc/Downloads/windows.shard-112.tfrecord', shuffle=1000)
# print(dataset)
# print(next(iter(dataset)).keys())

tf.Tensor(12182, shape=(), dtype=int64)


In [6]:
n = 0
for example in dataset:
    _ = example
    n += 1
print(n)

12182


95.171875

In [1]:
import pyBigWig
import numpy as np

In [2]:
print(pyBigWig.__version__)
print(np.__version__)

0.3.18
1.24.1
